In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

TensorFlow version: 2.20.0
Keras version: 3.12.0


In [2]:
class CustomDense(keras.layers.Layer):
    def __init__(self, units=32, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
    
    def build(self, input_shape):
        # Создание весов при известной форме входных данных
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="glorot_uniform",
            trainable=True,
            name="kernel"
        )
        self.b = self.add_weight(
            shape=(self.units,),
            initializer="zeros",
            trainable=True,
            name="bias"
        )
        super().build(input_shape)
    
    def call(self, inputs):
        # Прямое распространение
        output = tf.matmul(inputs, self.w) + self.b
        if self.activation is not None:
            output = self.activation(output)
        return output
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "units": self.units,
            "activation": keras.activations.serialize(self.activation)
        })
        return config

# Тестирование пользовательского слоя
print("=== Тестирование CustomDense слоя ===")
custom_layer = CustomDense(units=4, activation='relu')
x = tf.ones((2, 8))
y = custom_layer(x)
print(f"Входная форма: {x.shape}")
print(f"Выходная форма: {y.shape}")
print(f"Веса слоя: {len(custom_layer.weights)}")

=== Тестирование CustomDense слоя ===
Входная форма: (2, 8)
Выходная форма: (2, 4)
Веса слоя: 2


In [3]:
class AccumulatorLayer(keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def build(self, input_shape):
        # Создание нетренируемого веса для аккумуляции
        self.accumulator = self.add_weight(
            shape=(input_shape[-1],),
            initializer="zeros",
            trainable=False,  # Важно: веса не будут обучаться
            name="accumulator"
        )
        super().build(input_shape)
    
    def call(self, inputs):
        # Аккумуляция средних значений по батчу
        batch_mean = tf.reduce_mean(inputs, axis=0)
        self.accumulator.assign_add(batch_mean)
        return inputs  # Пропускаем вход без изменений

print("\n=== Тестирование AccumulatorLayer ===")
accum_layer = AccumulatorLayer()
x_test = tf.constant([[1.0, 2.0], [3.0, 4.0]], dtype=tf.float32)
output = accum_layer(x_test)
print(f"Входные данные: {x_test.numpy()}")
print(f"Аккумулятор после первого вызова: {accum_layer.accumulator.numpy()}")

# Второй вызов
x_test2 = tf.constant([[5.0, 6.0], [7.0, 8.0]], dtype=tf.float32)
output2 = accum_layer(x_test2)
print(f"Аккумулятор после второго вызова: {accum_layer.accumulator.numpy()}")

print(f"Все веса: {len(accum_layer.weights)}")
print(f"Тренируемые веса: {len(accum_layer.trainable_weights)}")
print(f"Нетренируемые веса: {len(accum_layer.non_trainable_weights)}")


=== Тестирование AccumulatorLayer ===
Входные данные: [[1. 2.]
 [3. 4.]]
Аккумулятор после первого вызова: [2. 3.]
Аккумулятор после второго вызова: [ 8. 10.]
Все веса: 1
Тренируемые веса: 0
Нетренируемые веса: 1


In [4]:
class CustomMLP(keras.layers.Layer):
    def __init__(self, hidden_units=[64, 32], output_units=10, dropout_rate=0.2, **kwargs):
        super().__init__(**kwargs)
        self.hidden_units = hidden_units
        self.output_units = output_units
        self.dropout_rate = dropout_rate
        
        # Создание подслоев
        self.hidden_layers = []
        for units in hidden_units:
            self.hidden_layers.append(
                CustomDense(units, activation='relu')
            )
        
        self.dropout = keras.layers.Dropout(dropout_rate)
        self.output_layer = CustomDense(output_units, activation='softmax')
    
    def call(self, inputs, training=False):
        x = inputs
        for layer in self.hidden_layers:
            x = layer(x)
        
        if training:
            x = self.dropout(x)
        
        return self.output_layer(x)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "hidden_units": self.hidden_units,
            "output_units": self.output_units,
            "dropout_rate": self.dropout_rate
        })
        return config

print("\n=== Тестирование CustomMLP ===")
mlp_model = CustomMLP(hidden_units=[128, 64], output_units=10)
x_sample = tf.random.normal((4, 28 * 28))
y_sample = mlp_model(x_sample)

print(f"Входная форма: {x_sample.shape}")
print(f"Выходная форма: {y_sample.shape}")
print(f"Общее количество весов: {len(mlp_model.weights)}")
print(f"Тренируемые веса: {len(mlp_model.trainable_weights)}")


=== Тестирование CustomMLP ===
Входная форма: (4, 784)
Выходная форма: (4, 10)
Общее количество весов: 6
Тренируемые веса: 6


In [5]:
class RegularizedDense(keras.layers.Layer):
    def __init__(self, units=32, l1_rate=1e-4, l2_rate=1e-4, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.l1_rate = l1_rate
        self.l2_rate = l2_rate
    
    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="he_normal",
            trainable=True,
            name="kernel"
        )
        self.b = self.add_weight(
            shape=(self.units,),
            initializer="zeros",
            trainable=True,
            name="bias"
        )
        super().build(input_shape)
    
    def call(self, inputs):
        # Добавление L1 и L2 регуляризации
        if self.l1_rate > 0:
            l1_loss = tf.reduce_sum(tf.abs(self.w)) * self.l1_rate
            self.add_loss(l1_loss)
        
        if self.l2_rate > 0:
            l2_loss = tf.reduce_sum(tf.square(self.w)) * self.l2_rate
            self.add_loss(l2_loss)
        
        return tf.matmul(inputs, self.w) + self.b
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "units": self.units,
            "l1_rate": self.l1_rate,
            "l2_rate": self.l2_rate
        })
        return config

print("\n=== Тестирование RegularizedDense ===")
reg_layer = RegularizedDense(units=8, l1_rate=1e-3, l2_rate=1e-3)
x_test = tf.ones((2, 16))

# Первый вызов для инициализации весов
_ = reg_layer(x_test)
print(f"Потери после первого вызова: {len(reg_layer.losses)}")
print(f"Значения потерь: {[loss.numpy() for loss in reg_layer.losses]}")

# Второй вызов (потери сбрасываются)
_ = reg_layer(x_test)
print(f"Потери после второго вызова: {len(reg_layer.losses)}")


=== Тестирование RegularizedDense ===
Потери после первого вызова: 2
Значения потерь: [np.float32(0.03391433), np.float32(0.01391584)]
Потери после второго вызова: 2


In [6]:
def create_custom_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    
    # Пользовательские слои
    x = CustomDense(128, activation='relu')(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = RegularizedDense(64, l1_rate=1e-4, l2_rate=1e-4)(x)
    x = keras.layers.ReLU()(x)
    x = keras.layers.Dropout(0.3)(x)
    x = CustomDense(32, activation='relu')(x)
    outputs = CustomDense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

print("\n=== Создание полной модели ===")
model = create_custom_model((784,), 10)
model.summary()

# Компиляция модели
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Модель успешно создана и скомпилирована!")


=== Создание полной модели ===



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ custom_dense_4 (CustomDense)         │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ regularized_dense_1                  │ (None, 64)                  │           8,256 │
│ (RegularizedDense)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu (ReLU)                         │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ custom_dense_5 (CustomDense)         │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ custom_dense_6 (CustomDense)         │ (None, 10)                  │             330 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 111,658 (436.16 KB)

 Trainable params: 111,402 (435.16 KB)

 Non-trainable params: 256 (1.00 KB)

Модель успешно создана и скомпилирована!


In [12]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

# Для Keras 3 используем keras.utils.register_keras_serializable
@keras.utils.register_keras_serializable()
class CustomDense(keras.layers.Layer):
    def __init__(self, units=32, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
    
    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="glorot_uniform",
            trainable=True,
            name="kernel"
        )
        self.b = self.add_weight(
            shape=(self.units,),
            initializer="zeros",
            trainable=True,
            name="bias"
        )
        super().build(input_shape)
    
    def call(self, inputs):
        output = tf.matmul(inputs, self.w) + self.b
        if self.activation is not None:
            output = self.activation(output)
        return output
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "units": self.units,
            "activation": keras.activations.serialize(self.activation)
        })
        return config

@keras.utils.register_keras_serializable()
class RegularizedDense(keras.layers.Layer):
    def __init__(self, units=32, l1_rate=1e-4, l2_rate=1e-4, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.l1_rate = l1_rate
        self.l2_rate = l2_rate
    
    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="he_normal",
            trainable=True,
            name="kernel"
        )
        self.b = self.add_weight(
            shape=(self.units,),
            initializer="zeros",
            trainable=True,
            name="bias"
        )
        super().build(input_shape)
    
    def call(self, inputs):
        if self.l1_rate > 0:
            l1_loss = tf.reduce_sum(tf.abs(self.w)) * self.l1_rate
            self.add_loss(l1_loss)
        
        if self.l2_rate > 0:
            l2_loss = tf.reduce_sum(tf.square(self.w)) * self.l2_rate
            self.add_loss(l2_loss)
        
        return tf.matmul(inputs, self.w) + self.b
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "units": self.units,
            "l1_rate": self.l1_rate,
            "l2_rate": self.l2_rate
        })
        return config

def create_custom_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    x = CustomDense(128, activation='relu')(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = RegularizedDense(64, l1_rate=1e-4, l2_rate=1e-4)(x)
    x = keras.layers.ReLU()(x)
    x = keras.layers.Dropout(0.3)(x)
    x = CustomDense(32, activation='relu')(x)
    outputs = CustomDense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Создаем и компилируем модель
print("=== Создание модели ===")
model = create_custom_model((784,), 10)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Структура модели:")
model.summary()

print("\n=== Тестирование пользовательских слоев ===")

# Тестирование CustomDense
print("1. Тестирование CustomDense:")
dense_layer = CustomDense(64, activation='relu', name='custom_dense_test')

# Инициализируем слой, вызвав его
x_test_dense = tf.random.normal((2, 32))
output_dense = dense_layer(x_test_dense)
print(f"Вход: {x_test_dense.shape}, Выход: {output_dense.shape}")
print(f"Количество весов: {len(dense_layer.weights)}")

print("\n2. Тестирование RegularizedDense:")
reg_layer = RegularizedDense(32, l1_rate=1e-3, l2_rate=1e-3)
output_reg = reg_layer(x_test_dense)
print(f"Вход: {x_test_dense.shape}, Выход: {output_reg.shape}")
print(f"Потери регуляризации: {len(reg_layer.losses)}")

print("\n=== Тестирование сериализации отдельных слоев ===")

# Сериализация CustomDense
config_dense = dense_layer.get_config()
print(f"Конфигурация CustomDense: {config_dense}")

# Восстановление из конфигурации
new_dense_layer = CustomDense.from_config(config_dense)
_ = new_dense_layer(x_test_dense)  # Инициализация
print(f"Восстановленный слой: units={new_dense_layer.units}")

print("\n=== Тестирование сериализации полной модели ===")

# Метод 1: Через get_config() и from_config()
model_config = model.get_config()

custom_objects = {
    'CustomDense': CustomDense,
    'RegularizedDense': RegularizedDense
}

reconstructed_model = keras.Model.from_config(model_config, custom_objects=custom_objects)
print("✅ Модель успешно восстановлена через from_config()!")

print("\n=== Проверка работоспособности моделей ===")
# Создаем тестовые данные правильной формы
x_test = tf.random.normal((3, 784))

# Тестируем оригинальную модель
original_output = model(x_test)
print(f"Оригинальная модель: выходная форма {original_output.shape}")

# Тестируем восстановленную модель
reconstructed_output = reconstructed_model(x_test)
print(f"Восстановленная модель: выходная форма {reconstructed_output.shape}")

# Сравниваем выходы
are_identical = tf.reduce_all(tf.abs(original_output - reconstructed_output) < 1e-6).numpy()
print(f"Выходы идентичны: {are_identical}")

print("\n=== Тестирование сохранения и загрузки ===")

# Сохраняем модель
model_path = 'custom_model.keras'
model.save(model_path)
print(f"✅ Модель сохранена в {model_path}")

# Загружаем модель
loaded_model = keras.models.load_model(
    model_path,
    custom_objects=custom_objects
)
print("✅ Модель успешно загружена!")

# Проверяем загруженную модель
loaded_output = loaded_model(x_test)
print(f"Загруженная модель: выходная форма {loaded_output.shape}")

are_identical_loaded = tf.reduce_all(tf.abs(original_output - loaded_output) < 1e-6).numpy()
print(f"Загруженная модель идентична оригиналу: {are_identical_loaded}")

print("\n=== Тестирование обучения восстановленной модели ===")

# Создаем простые синтетические данные
x_train = tf.random.normal((100, 784))
y_train = tf.random.uniform((100,), minval=0, maxval=10, dtype=tf.int32)

# Компилируем и обучаем восстановленную модель
reconstructed_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Обучение восстановленной модели (1 эпоха)...")
history = reconstructed_model.fit(
    x_train, y_train,
    epochs=1,
    batch_size=16,
    verbose=1,
    validation_split=0.2
)

print("✅ Обучение завершено успешно!")

# Очистка
if os.path.exists(model_path):
    os.remove(model_path)
    print(f"🗑️ Временный файл {model_path} удален")



TensorFlow version: 2.20.0
Keras version: 3.12.0
=== Создание модели ===
Структура модели:


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ custom_dense_10 (CustomDense)        │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ regularized_dense_4                  │ (None, 64)                  │           8,256 │
│ (RegularizedDense)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_2 (ReLU)                       │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ custom_dense_11 (CustomDense)        │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ custom_dense_12 (CustomDense)        │ (None, 10)                  │             330 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 111,658 (436.16 KB)

 Trainable params: 111,402 (435.16 KB)

 Non-trainable params: 256 (1.00 KB)


=== Тестирование пользовательских слоев ===
1. Тестирование CustomDense:
Вход: (2, 32), Выход: (2, 64)
Количество весов: 2

2. Тестирование RegularizedDense:
Вход: (2, 32), Выход: (2, 32)
Потери регуляризации: 2

=== Тестирование сериализации отдельных слоев ===
Конфигурация CustomDense: {'name': 'custom_dense_test', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'units': 64, 'activation': 'relu'}
Восстановленный слой: units=64

=== Тестирование сериализации полной модели ===
✅ Модель успешно восстановлена через from_config()!

=== Проверка работоспособности моделей ===
Оригинальная модель: выходная форма (3, 10)
Восстановленная модель: выходная форма (3, 10)
Выходы идентичны: False

=== Тестирование сохранения и загрузки ===
✅ Модель сохранена в custom_model.keras
✅ Модель успешно загружена!
Загруженная модель: выходная форма (3, 10)
Загруженная модель идентична оригиналу: True

=== Тестирование об